In [ ]:
from dataloader import load_citation
import time
start_time = time.time()
dataf = "../data/"
norm_type = "SymNorm_tildeA"
#norm_type = "sym_normalized_A"
original_graph, L, features, labels, idx_train,idx_val, idx_test = load_citation(dataf,"cora",norm_type=norm_type,cuda=False, identity_features=False)
name = "cora"
print(time.time() - start_time)

In [ ]:
import time
import argparse
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
from models import GCN
from sklearn.metrics import f1_score
from utils import accuracy, set_seed


model_path = "../models/gcn_cora_state.pkt"
#model_path = "../models/mlp_cora_state.pkt"
#torch.save(model.state_dict(), model_path)
args = [1433,16,7,0.5]
#args = [3703,16,6,0.5]
#args = [500, 16, 3, 0.5]
#args = [1222, 16, 2, 0.5]
kwargs = {
    "bias": True,
}
model = GCN(*args,**kwargs)
model.load_state_dict(torch.load(model_path, map_location='cpu'))

model.eval()
output = model(features,L)
loss_test = F.nll_loss(output[idx_test],labels[idx_test])
acc_test = accuracy(output[idx_test],labels[idx_test])
print(loss_test.item(), acc_test.item())



In [ ]:
print(output)

In [ ]:
import argparse
import os

import sklearn.metrics as metrics

from tensorboardX import SummaryWriter

import pickle
import shutil
import torch

import models
#import utils.io_utils as io_utils
#import utils.parser_utils as parser_utils
#import utils.graph_utils as graph_utils
from explainer import explain
""" parser_utils.py

    Parsing utilities.
"""
import argparse
import numpy as np
import os
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline  
def parse_optimizer(parser):
    '''Set optimizer parameters'''
    opt_parser = parser.add_argument_group()
    opt_parser.add_argument('--opt', dest='opt', type=str,
            help='Type of optimizer')
    opt_parser.add_argument('--opt-scheduler', dest='opt_scheduler', type=str,
            help='Type of optimizer scheduler. By default none')
    opt_parser.add_argument('--opt-restart', dest='opt_restart', type=int,
            help='Number of epochs before restart (by default set to 0 which means no restart)')
    opt_parser.add_argument('--opt-decay-step', dest='opt_decay_step', type=int,
            help='Number of epochs before decay')
    opt_parser.add_argument('--opt-decay-rate', dest='opt_decay_rate', type=float,
            help='Learning rate decay ratio')
    opt_parser.add_argument('--lr', dest='lr', type=float,
            help='Learning rate.')
    opt_parser.add_argument('--clip', dest='clip', type=float,
            help='Gradient clipping.')


def arg_parse():
    parser = argparse.ArgumentParser(description="GNN Explainer arguments.")
    io_parser = parser.add_mutually_exclusive_group(required=False)
    io_parser.add_argument("--dataset", dest="dataset", help="Input dataset.")
    benchmark_parser = io_parser.add_argument_group()
    benchmark_parser.add_argument(
        "--bmname", dest="bmname", help="Name of the benchmark dataset"
    )
    io_parser.add_argument("--pkl", dest="pkl_fname", help="Name of the pkl data file")

    parse_optimizer(parser)

    parser.add_argument("--clean-log", action="store_true", help="If true, cleans the specified log directory before running.")
    parser.add_argument("--logdir", dest="logdir", help="Tensorboard log directory")
    parser.add_argument("--ckptdir", dest="ckptdir", help="Model checkpoint directory")
    parser.add_argument("--cuda", dest="cuda", help="CUDA.")
    parser.add_argument(
        "--gpu",
        dest="gpu",
        action="store_const",
        const=True,
        default=False,
        help="whether to use GPU.",
    )
    parser.add_argument(
        "--epochs", dest="num_epochs", type=int, help="Number of epochs to train."
    )
    parser.add_argument(
        "--hidden-dim", dest="hidden_dim", type=int, help="Hidden dimension"
    )
    parser.add_argument(
        "--output-dim", dest="output_dim", type=int, help="Output dimension"
    )
    parser.add_argument(
        "--num-gc-layers",
        dest="num_gc_layers",
        type=int,
        help="Number of graph convolution layers before each pooling",
    )
    parser.add_argument(
        "--bn",
        dest="bn",
        action="store_const",
        const=True,
        default=False,
        help="Whether batch normalization is used",
    )
    parser.add_argument("--dropout", dest="dropout", type=float, help="Dropout rate.")
    parser.add_argument(
        "--nobias",
        dest="bias",
        action="store_const",
        const=False,
        default=True,
        help="Whether to add bias. Default to True.",
    )
    parser.add_argument(
        "--no-writer",
        dest="writer",
        action="store_const",
        const=False,
        default=False,
        help="Whether to add bias. Default to True.",
    )
    # Explainer
    parser.add_argument("--mask-act", dest="mask_act", type=str, help="sigmoid, ReLU.")
    parser.add_argument(
        "--mask-bias",
        dest="mask_bias",
        action="store_const",
        const=True,
        default=False,
        help="Whether to add bias. Default to True.",
    )
    parser.add_argument(
        "--explain-node", dest="explain_node", type=int, help="Node to explain."
    )
    parser.add_argument(
        "--graph-idx", dest="graph_idx", type=int, help="Graph to explain."
    )
    parser.add_argument(
        "--graph-mode",
        dest="graph_mode",
        action="store_const",
        const=True,
        default=False,
        help="whether to run Explainer on Graph Classification task.",
    )
    parser.add_argument(
        "--multigraph-class",
        dest="multigraph_class",
        type=int,
        help="whether to run Explainer on multiple Graphs from the Classification task for examples in the same class.",
    )
    parser.add_argument(
        "--multinode-class",
        dest="multinode_class",
        type=int,
        help="whether to run Explainer on multiple nodes from the Classification task for examples in the same class.",
    )
    parser.add_argument(
        "--align-steps",
        dest="align_steps",
        type=int,
        help="Number of iterations to find P, the alignment matrix.",
    )

    parser.add_argument(
        "--method", dest="method", type=str, help="Method. Possible values: base, att."
    )
    parser.add_argument(
        "--name-suffix", dest="name_suffix", help="suffix added to the output filename"
    )
    parser.add_argument(
        "--explainer-suffix",
        dest="explainer_suffix",
        help="suffix added to the explainer log",
    )

    # TODO: Check argument usage
    parser.set_defaults(
        logdir="log",
        ckptdir="ckpt",
        dataset="syn1",
        opt="adam",  
        opt_scheduler="none",
        cuda="0",
        lr=0.1,
        clip=2.0,
        batch_size=20,
        num_epochs=100,
        hidden_dim=20,
        output_dim=20,
        num_gc_layers=2,
        dropout=0.0,
        method="base",
        name_suffix="",
        explainer_suffix="",
        align_steps=1000,
        explain_node=None,
        graph_idx=-1,
        mask_act="sigmoid",
        multigraph_class=-1,
        multinode_class=-1,
    )
    return parser.parse_args(args=[])

In [ ]:
# Load a configuration
prog_args = arg_parse()
cg_dict = {}
cg_dict["adj"] = original_graph.to_dense().unsqueeze(0).detach().numpy()
cg_dict["feat"] = features.unsqueeze(0).detach().numpy()
cg_dict["label"] = labels.unsqueeze(0).detach().numpy()
cg_dict["train_idx"] = idx_train.unsqueeze(0).detach().numpy()
cg_dict["pred"] = output.unsqueeze(0).detach().numpy()

In [ ]:
print(cg_dict["adj"].shape, cg_dict["feat"].shape, cg_dict["label"].shape, cg_dict["train_idx"].shape, cg_dict["pred"].shape)

In [ ]:

# Create explainer
explainer = explain.Explainer(
    model=model,
    adj=cg_dict["adj"],
    feat=cg_dict["feat"],
    label=cg_dict["label"],
    pred=cg_dict["pred"],
    train_idx=cg_dict["train_idx"],
    args=prog_args,
    writer=None,
    print_training=False,
    graph_mode=False,
    graph_idx=0,
)
print(prog_args)

In [ ]:
'''
adj = cg_dict["adj"]
node_idx_new, sub_adj, sub_feat, sub_label, neighbors = explainer.extract_neighborhood(explain_node_idx)
neighbors_dict = {}
for i in range(len(neighbors)):
    neighbors_dict[i] = neighbors[i]
def constructNewNeighborDict(node_list, neighbor_dict):
    new_neighbor_dict = {}
    for node_id in node_list:
        new_neighbor_dict[node_id] = neighbor_dict[node_id]
    return new_neighbor_dict
for m in masked_adj: 

    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,5))
    #plt.title(str(m))
    
    # Full adjacency
    ax1.set_title('Full Graph')
    #ax1.imshow(adj)
    #G1 = nx.from_numpy_array(adj[0])
    #G1.remove_nodes_from(list(nx.isolates(G1)))
    #nx.draw(G, ax=ax3)
    #nx.draw(G1, ax=ax1)
    
    
    
    # Filtered adjacency
    f = filter_adj(m,0.3)
    #ax2.set_title('Sub neighbor')
    #ax2.imshow(f);
    
    G2 = nx.from_numpy_array(sub_adj)
    G2.remove_nodes_from(list(nx.isolates(G2)))
    nx.draw_networkx(G2, ax=ax2, labels=constructNewNeighborDict(G2.nodes,neighbors_dict))
    
    # Plot subgraph
    ax3.set_title("Explanation Sub graph")
    G = nx.from_numpy_array(f)
    G.remove_nodes_from(list(nx.isolates(G)))
    
    
    #nx.draw(G, ax=ax3)
    nx.draw_networkx(G, ax=ax3, labels=constructNewNeighborDict(G.nodes,neighbors_dict))
    '''
''''''

In [ ]:
explain_node_idx = 1892
# explain a set of nodes
masked_adj = explainer.explain_nodes_gnn_stats(
    [explain_node_idx], prog_args
) 


In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
def show_adjacency_full(mask, ax=None):
    adj = np.load(os.path.join(logdir, expdir, mask), allow_pickle=True)
    if ax is None:
        plt.figure()
        plt.imshow(adj);
    else:
        ax.imshow(adj)
    return adj
def filter_adj(adj,threshold=0.8):
    filt_adj = adj.copy()
    filt_adj[adj<threshold] = 0
    return filt_adj
adj = cg_dict["adj"]
node_idx_new, sub_adj, sub_feat, sub_label, neighbors = explainer.extract_neighborhood(explain_node_idx)
print(neighbors)
neighbors_dict = {}
for i in range(len(neighbors)):
    neighbors_dict[i] = neighbors[i]
def constructNewNeighborDict(node_list, neighbor_dict):
    new_neighbor_dict = {}
    for node_id in node_list:
        new_neighbor_dict[node_id] = neighbor_dict[node_id]
    return new_neighbor_dict
ground_truth_label = cg_dict["label"][0].tolist()
preds = cg_dict["pred"][0].argmax(1).tolist()
TF = []
for i in range(len(ground_truth_label)):
    if preds[i] == ground_truth_label[i]:
        TF.append(1)
    else:
        TF.append(0)
#print(TF)
'''
ground_truth_color = []
prediction_color = []
TF_color = []
for i in range(len(ground_truth_label)):
    ground_truth_color.append(d3_10color[ground_truth_label[i]])
    prediction_color.append(d3_10color[preds[i]])
    TF_color.append(tf_color[TF[i]])
'''
# EDIT THIS INDEX
MASK_IDX = 0
# EDIT THIS INDEX

m = masked_adj[MASK_IDX]
d3_10color = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b", "#e377c2", "#7f7f7f", "#bcbd22", "#17becf"]
tf_color = ["#DC143C","#498B77"]
def constructColorList(nodelist, labellist, colorlist):
    new_colorlist = []
    for node_idx in nodelist:
        node_id = nodelist[node_idx]
        label = labellist[node_id]
        new_colorlist.append(colorlist[label])
        #if node_id == 1708:
        #print(node_id, label, colorlist[label])
    return new_colorlist
def constructNodeSizeList(nodelist, explain_node_idx):
    sizelist = []
    for node_idx in nodelist:
        node_id = nodelist[node_idx]
        if node_id == explain_node_idx:
            sizelist.append(500)
        else:
            sizelist.append(300)
    return sizelist
@interact(thresh=widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.01))
def plot_interactive(thresh=0.5):
    f = filter_adj(m,thresh)
    G = nx.from_numpy_array(f)
    G.remove_nodes_from(list(nx.isolates(G)))
    pos = nx.spring_layout(G)
    #G2 = nx.from_numpy_array(sub_adj)
    #G2.remove_nodes_from(list(nx.isolates(G2)))
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,5))
    plt.title("Explanation Subgraph threshold:{}".format(thresh))
    
    ax1.set_title('Ground Truth ')
    real_label = constructNewNeighborDict(G.nodes,neighbors_dict)
    nx.draw_networkx(G, pos=pos, ax=ax1, labels=real_label, 
                     node_color=constructColorList(real_label,ground_truth_label,d3_10color)
                    ,node_size=constructNodeSizeList(real_label,explain_node_idx))

    # Filtered adjacency
    
    ax2.set_title('Model Prediction')
    nx.draw_networkx(G, pos=pos,ax=ax2, labels=real_label, 
                     node_color=constructColorList(real_label,preds,d3_10color)
                    ,node_size=constructNodeSizeList(real_label,explain_node_idx))
    
    # Plot subgraph
    ax3.set_title("T/F Explain Node Id:{} threshold:{}".format(explain_node_idx,thresh))
    nx.draw_networkx(G, pos=pos, ax=ax3, labels=real_label, 
                     node_color=constructColorList(real_label,TF,tf_color)
                    ,node_size=constructNodeSizeList(real_label,explain_node_idx))
    
